In [12]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataloader import get_loaders
from models import CNNRegressor, CNNClassifier, VGGRegressor

In [13]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [24]:
import os
directory = "trained_models"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory '{directory}' created.")
else:
    print(f"Directory '{directory}' already exists.")

Directory 'trained_models' already exists.


### Regressor Modelling

The given task is such that we are given an input image, and we are predicting the sum of the numbers represented in the image. If there is no guarantee as to whether the test set contains exactly 4 numbers, the typical classifier approach will likely fail. As a result, we can modify the network to act as a regressor with a single output and the output value being rounded to get an integer representation. Hence, during training, the model will use MSE Loss (typically used for regression tasks) and will round the number to closest integer during inference to calculate accuracy. 

Note that as a result of this approach, a low MSE Loss need not translate to a good accuracy. Consider for example, our model predicts 24.6 as an output while the ground truth is 24. In this case, the MSE Loss will be low, but the accuracy need not be as rounding changes the output. Also note that it is not possible to train the model based on rounded outputs as rounding is a step function with zero gradient almost everywhere making it useless for learning. 

In [14]:
def train_model(train_loader, val_loader, model, criterion, optimizer, epochs, device, type="regressor"):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            if type=="classifier":
                labels = labels.to(torch.long)

            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_acc = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                if type=="classifier":
                    outputs = F.softmax(outputs, dim=-1)
                    outputs = torch.argmax(outputs, dim=1)
                accuracy = (torch.sum(torch.round(outputs)==labels)*100/len(labels)).item()
                val_acc += accuracy

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Accuracy: {val_acc/len(val_loader):.4f}")

In [15]:
# Hyperparameters
device = "cuda" if torch.cuda.is_available() else "cpu"
train_fraction = 0.8
batch_size = 64
seed = 42
epochs = 15
learning_rate = 1e-3
seed_everything(seed)

In [16]:
model = CNNRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader, val_loader = get_loaders(train_fraction, batch_size, seed)
train_model(train_loader, val_loader, model, criterion, optimizer, epochs, device)

Epoch 1/15, Train Loss: 34.8493, Val Accuracy: 8.4386
Epoch 2/15, Train Loss: 21.6001, Val Accuracy: 9.4692
Epoch 3/15, Train Loss: 15.5664, Val Accuracy: 10.5773
Epoch 4/15, Train Loss: 12.6231, Val Accuracy: 11.4971
Epoch 5/15, Train Loss: 10.8117, Val Accuracy: 11.9293
Epoch 6/15, Train Loss: 9.6357, Val Accuracy: 9.6133
Epoch 7/15, Train Loss: 9.2238, Val Accuracy: 12.9266
Epoch 8/15, Train Loss: 8.2191, Val Accuracy: 12.3670
Epoch 9/15, Train Loss: 7.4315, Val Accuracy: 12.0124
Epoch 10/15, Train Loss: 7.4809, Val Accuracy: 12.1786
Epoch 11/15, Train Loss: 6.2272, Val Accuracy: 11.9238
Epoch 12/15, Train Loss: 5.8341, Val Accuracy: 12.6274
Epoch 13/15, Train Loss: 6.3204, Val Accuracy: 12.9045
Epoch 14/15, Train Loss: 5.6288, Val Accuracy: 12.8823
Epoch 15/15, Train Loss: 5.2671, Val Accuracy: 10.6494


In [17]:
## Calculating overall accuracy of the validation set
model.eval()
total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Regressor on Validation Split: {accuracy:.2f}%")

total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Regressor on Training Split: {accuracy:.2f}%")

Overall Accuracy of Regressor on Validation Split: 10.65%
Overall Accuracy of Regressor on Training Split: 18.16%


In [18]:
torch.save(model.state_dict(), f"./trained_models/cnnregressor_{seed}.pt")

### Classifier Modelling

When we are assuming that the input image will always have 4 numbers as that is the only value appearing in the dataset. In that case, we can model the network as a classifier with the outputs ranging from 0 to 36. For this case, we can use the cross entropy loss which is commonly used for classification task. Empirically we observe that the model performance on the validation split is lower for the classifier than it is for the regressor. This might be because regression operates in the continuous space (just like the input) and this possibly allows for a smoother optimization. It is not possible to verify why the regressor works better than the classifier case. 

In [19]:
model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader, val_loader = get_loaders(train_fraction, batch_size, seed)
train_model(train_loader, val_loader, model, criterion, optimizer, epochs, device, type="classifier")

Epoch 1/15, Train Loss: 3.1989, Val Accuracy: 6.9204
Epoch 2/15, Train Loss: 3.0899, Val Accuracy: 7.7405
Epoch 3/15, Train Loss: 2.9005, Val Accuracy: 9.2531
Epoch 4/15, Train Loss: 2.7468, Val Accuracy: 9.2586
Epoch 5/15, Train Loss: 2.5727, Val Accuracy: 8.6990
Epoch 6/15, Train Loss: 2.3447, Val Accuracy: 8.7600
Epoch 7/15, Train Loss: 2.0550, Val Accuracy: 7.9566
Epoch 8/15, Train Loss: 1.7215, Val Accuracy: 8.1449
Epoch 9/15, Train Loss: 1.3895, Val Accuracy: 8.3666
Epoch 10/15, Train Loss: 1.1216, Val Accuracy: 7.9289
Epoch 11/15, Train Loss: 0.9226, Val Accuracy: 7.8125
Epoch 12/15, Train Loss: 0.6945, Val Accuracy: 7.6186
Epoch 13/15, Train Loss: 0.5235, Val Accuracy: 6.7764
Epoch 14/15, Train Loss: 0.4084, Val Accuracy: 7.0368
Epoch 15/15, Train Loss: 0.3506, Val Accuracy: 7.6961


In [20]:
## Calculating overall accuracy of the validation set
model.eval()
total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        outputs = F.softmax(outputs, dim=-1)
        outputs = torch.argmax(outputs, dim=1)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Classifer on Validation Split: {accuracy:.2f}%")

total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        outputs = F.softmax(outputs, dim=-1)
        outputs = torch.argmax(outputs, dim=1)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Classifer on Training Split: {accuracy:.2f}%")

Overall Accuracy of Classifer on Validation Split: 7.70%
Overall Accuracy of Classifer on Training Split: 68.08%


In [21]:
torch.save(model.state_dict(), f"./trained_models/cnnclassifier_{seed}.pt")

The same behaviour was observed across seeds (0, 10, 42), indicating a consistent behaviour. Also note that early stopping based on validation loss has not been implemented for the models (even though the validation accuracy is higher at some early stages) to ensure the train loss is sufficiently low. Training for further epochs leads to a decrease in the validation accuracy indicating overfitting. Hence, the number of epochs was chosen to be 15. Note that the training accuracy is very high for the classifier approach, but the validation is not, indicating a possibility that this approach is overfitting. 

### Transfer Learning With VGG

Owing to the success of the regressor approach, a third type of network was created using transfer learning using VGG (a popular CNN network which achieves fairly high accuracy across various tasks). The convolutional backbone and the average pooling layers of the network are frozen and a classifier is added on top which has one output as the previous CNN regressor case. Note that a convolutional layer is also prepended to ensure that the MNIST dataset with one channel is compatible with the VGG network that is trained on 3 channel RGB images. This approach is computationally more expensive, but was tried out to see whether the pretraining of the VGG leads to better results.  

In [22]:
model = VGGRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader, val_loader = get_loaders(train_fraction, batch_size, seed)
train_model(train_loader, val_loader, model, criterion, optimizer, epochs, device)

KeyboardInterrupt: 

In [22]:
## Calculating overall accuracy of the validation set
model.eval()
total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Regressor on Validation Split: {accuracy:.2f}%")

total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Regressor on Training Split: {accuracy:.2f}%")

Overall Accuracy of Regressor on Validation Split: 10.23%
Overall Accuracy of Regressor on Training Split: 9.98%


In [23]:
torch.save(model.state_dict(), f"./trained_models/vggregressor_{seed}.pt")

### Results Across Seeds

#### Validation Accuracy

| Seed | CNN Regressor | CNN Classifier | VGG Regressor |
|------|---------------|----------------|---------------|
| 0    | 11.42%        | 6.70%          | 10.23%        |
| 10   | 10.92%        | 8.00%          | 0             |
| 42   | 10.65%        | 7.70%          | 9.12%         |

#### Training Accuracy

| Seed | CNN Regressor | CNN Classifier | VGG Regressor |
|------|---------------|----------------|---------------|
| 0    | 21.90%        | 52.36%         | 9.98%         |
| 10   | 22.22%        | 76.92%         | 0             |
| 42   | 18.16%        | 68.08%         | 9.29%         |